In [1]:
history = []
if history:
    print(1)
else:
    print(2)

2


In [1]:
import psycopg2
from psycopg2 import Error
import re 
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,DocumentSummaryIndex
from llama_index.core import PromptTemplate,get_response_synthesizer
from llama_index.core.settings import Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.deepseek import DeepSeek
import chromadb
import os

d:\python_envs\ai_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
from util.chat_util import initialize_llamaindex
documents = SimpleDirectoryReader(input_dir="D:\\ai_ztf\\fastapi_project\\data\\").load_data()
llm, embed_model = initialize_llamaindex(
    deepseekapi="sk-5f2880952eb543a59d02d1015dcdd8e1"  # 替换为您的DeepSeek API密钥
    )
for doc in documents:
    # 将原始doc_id存储在metadata中
    doc.metadata['original_doc_id'] = doc.doc_id
##创建摘要索引，自定义摘要内容
def generate_summary_index():
    #初始化 Chroma 客户端
    chroma_client = chromadb.Client()
    chroma_collection = chroma_client.get_or_create_collection("text_doc1")
    # 创建向量存储orm
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    summary_tmpl = PromptTemplate(
    """请用1-2句话总结以下文档，并提出1-2个该文档能回答的用户问题：\n
    {context_str}
    回答的格式形如：
    **文档摘要**
    --你的摘要内容
    **可以回答的问题**
    --问题列表
    """)
    # LlamaIndex ≥0.10.x 把 summary_template 收进 response_synthesizer
    resp_synth = get_response_synthesizer(
        response_mode="tree_summarize",    
        summary_template=summary_tmpl
    )
    # 构建 DocumentSummaryIndex
    
    doc_sum_index = DocumentSummaryIndex.from_documents(
        documents,
        response_synthesizer=resp_synth,
        vector_store=vector_store
    )
    return doc_sum_index
summary_index = generate_summary_index()

current doc id: db196138-650f-4c06-832d-f31b9d8ee8da
current doc id: 8053f065-5e4a-42f8-924a-7ca601fb4485


In [14]:
summary_index.docstore.get_node()

TypeError: BaseDocumentStore.get_node() missing 1 required positional argument: 'node_id'

IndexDocumentSummary(index_id='69f193a1-de89-44ab-8589-77c35ed21b05', summary=None, summary_id_to_node_ids={'ac6b2fc1-7462-47a8-9dea-890641e88067': ['4736d4c2-81fc-42c4-b214-efa790f53b3e', 'dca67fbc-357a-42cd-bdcf-a8ba63050c6a', 'a900e0c5-399e-4304-84ff-a0744fd01d87'], '436c9c1e-a0e5-4707-9653-f4fa7b960e10': ['249ba425-eddf-4cb1-82db-ad0c036e25f6', '1d3ce589-af44-4061-8f71-66781e9d30c9', '8dc9f615-eca8-44c6-9b18-f766ad1c8c66']}, node_id_to_summary_id={'4736d4c2-81fc-42c4-b214-efa790f53b3e': 'ac6b2fc1-7462-47a8-9dea-890641e88067', 'dca67fbc-357a-42cd-bdcf-a8ba63050c6a': 'ac6b2fc1-7462-47a8-9dea-890641e88067', 'a900e0c5-399e-4304-84ff-a0744fd01d87': 'ac6b2fc1-7462-47a8-9dea-890641e88067', '249ba425-eddf-4cb1-82db-ad0c036e25f6': '436c9c1e-a0e5-4707-9653-f4fa7b960e10', '1d3ce589-af44-4061-8f71-66781e9d30c9': '436c9c1e-a0e5-4707-9653-f4fa7b960e10', '8dc9f615-eca8-44c6-9b18-f766ad1c8c66': '436c9c1e-a0e5-4707-9653-f4fa7b960e10'}, doc_id_to_summary_id={'2d1d2be1-3037-4a1c-9c2e-e884b8f52bb4': '

dict_items([('2d1d2be1-3037-4a1c-9c2e-e884b8f52bb4', 'ac6b2fc1-7462-47a8-9dea-890641e88067'), ('0e363d46-397f-4def-939a-c2589a02091a', '436c9c1e-a0e5-4707-9653-f4fa7b960e10')])

In [1]:
##-----DONE-----
def show_doc_summaries(index: DocumentSummaryIndex):
    """打印索引中每篇文档的摘要"""
    struct = index.index_struct                 # IndexDocumentSummary 对象
    ds = index.docstore
    #这个是获取的doc_id和summary_id
    for doc_id, summary_node_id in struct.doc_id_to_summary_id.items():
        summary_node = ds.get_node(summary_node_id)
        print(f"\n=== 文档 {doc_id} 的摘要 ===")
        print(summary_node.get_text())

# 调用函数显示摘要
show_doc_summaries(summary_index)


NameError: name 'DocumentSummaryIndex' is not defined

In [ ]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

# 示例数据（假设documents已存在，且每个doc有doc_id）

def generate_simple_index():
    # 按换行符分割，并禁用自动句子分割
    splitter = SentenceSplitter(
        separator="\n",  # 强制按换行符切分
        chunk_size=1024,  # 可选：防止单个node过长
        chunk_overlap=0,  # 无重叠
        paragraph_separator="\n",  # 明确段落分隔符
        secondary_chunking_regex="\n",  # 二级分割同样用换行符
    )

    # 分割并保留doc_id
    nodes = []
    for doc in documents:
        # 对当前doc生成nodes
        doc_nodes = splitter.get_nodes_from_documents([doc])
        # 为每个node添加原doc_id到metadata
        for node in doc_nodes:
            node.metadata["doc_id"] = doc.doc_id
        nodes.extend(doc_nodes)

    # 验证输出
    for node in nodes:
        print(f"Node ID: {node.node_id}, Doc ID: {node.metadata['doc_id']}, Text: {node.text[:20]}...")

    # 创建索引（新版无需ServiceContext）
    index = VectorStoreIndex(nodes)
    return index

Node ID: 19c0c560-b7de-4a02-affa-931cf66ebe49, Doc ID: db196138-650f-4c06-832d-f31b9d8ee8da, Text: 我们在上海特区第二监狱里关了两天。到廿六...
Node ID: 13c01a0c-a6e3-4443-bce9-687c2c472f2a, Doc ID: db196138-650f-4c06-832d-f31b9d8ee8da, Text: 到了地方法院之后，我和章先生被几个法警押...
Node ID: 0182b86a-f8ac-482e-a93b-c7690a7dba98, Doc ID: db196138-650f-4c06-832d-f31b9d8ee8da, Text: 总算优待，原来八个人住的一间囚室，经他们...
Node ID: feb44ea6-0a40-4af3-a7a7-280c6d16c23c, Doc ID: 8053f065-5e4a-42f8-924a-7ca601fb4485, Text: 在上海地方法院的看守所里只关了一天一夜，...
Node ID: 51e44485-c54a-42e8-812e-dd07f23056ea, Doc ID: 8053f065-5e4a-42f8-924a-7ca601fb4485, Text: 这时沈先生等四位已押在公安局三四天了，我...
Node ID: 94ffe1c8-c62b-422a-83e8-ac129d37de11, Doc ID: 8053f065-5e4a-42f8-924a-7ca601fb4485, Text: 但是他们对于职务还是不得不奉行故事的。他...


In [37]:
# 打印所有文档的ID
for doc in documents:
    print(f"文档ID: {doc.doc_id}")


文档ID: db196138-650f-4c06-832d-f31b9d8ee8da
文档ID: 8053f065-5e4a-42f8-924a-7ca601fb4485


In [40]:
##根据用户提问，使用摘要索引获得相关文档
def get_docs_from_summaryindex(doc_sum_index,query):
    retriever = doc_sum_index.as_retriever(similarity_top_k=3)
    # 手动进行检索（结果为 NodeWithScore 对象）
    retrieved_nodes = retriever.retrieve(query)
     # 使用集合去重
    seen_doc_ids = set()
    result = []
    for node in retrieved_nodes:
        ref_doc_id = node.node.ref_doc_id
        # 只添加未见过的文档ID
        if ref_doc_id not in seen_doc_ids:
            seen_doc_ids.add(ref_doc_id)
            result.append({
                'ref_doc_id': ref_doc_id,
                'score': node.score,  # 相关性分数
                'text': node.node.text[:100] + '...',  # 文本预览
            })
    
    return result
res = get_docs_from_summaryindex(summary_index,"作者和章先生在地方法院经历了什么？")
selected_doc_ids = [r['ref_doc_id'] for r in res]# 创建检索器，检索器只能检索信息，不需要进一步创建搜索或对话引擎
def get_final_nodes_text():
    retriever = index.as_retriever(similarity_top_k=5)
    # 获取相关文档
    retrieved_docs = retriever.retrieve("作者和章先生在地方法院经历了什么？")
    res_text = ''
    for nodeWithScore in retrieved_docs:
        if nodeWithScore.node.metadata['original_doc_id'] in selected_doc_ids:
            res += nodeWithScore.node.text + '\n\n'
    return res_text

In [41]:
res

[{'ref_doc_id': 'db196138-650f-4c06-832d-f31b9d8ee8da',
  'score': None,
  'text': '我们在上海特区第二监狱里关了两天。到廿六日那一天下午，我们在四点半后照例“归号”,铁格子门锁上了，我们两人照常在那小囚室里拥挤着。七点钟左右，忽有一个看守进来说“接见”。所谓“接见”,是监狱里允许囚犯...'},
 {'ref_doc_id': '8053f065-5e4a-42f8-924a-7ca601fb4485',
  'score': None,
  'text': '在上海地方法院的看守所里只关了一天一夜，廿七日下午六点钟左右，又像老鼠被猫衔在嘴里奔窜着，由地方法院转解到公安局里去。我们先由看守所里被提出来，重新光顾待审室一次，不过这时我和章先生两个人同被押进一间...'}]

In [42]:
selected_doc_ids = [r['ref_doc_id'] for r in res]# 创建检索器，检索器只能检索信息，不需要进一步创建搜索或对话引擎
def get_final_nodes_text():
    retriever = index.as_retriever(similarity_top_k=5)
    # 获取相关文档
    retrieved_docs = retriever.retrieve("作者和章先生在地方法院经历了什么？")
    res = ''
    for nodeWithScore in retrieved_docs:
        if nodeWithScore.node.metadata['original_doc_id'] in selected_doc_ids:
            res += nodeWithScore.node.text + '\n\n'
    return res

In [43]:
selected_doc_ids

['db196138-650f-4c06-832d-f31b9d8ee8da',
 '8053f065-5e4a-42f8-924a-7ca601fb4485']

In [45]:
# 创建检索器，检索器只能检索信息，不需要进一步创建搜索或对话引擎
def get_final_nodes_text():
    retriever = index.as_retriever(similarity_top_k=5)
    # 获取相关文档
    retrieved_docs = retriever.retrieve("作者和章先生在地方法院经历了什么？")
    res = ''
    for nodeWithScore in retrieved_docs:
        if nodeWithScore.node.metadata['original_doc_id'] in selected_doc_ids:
            res += nodeWithScore.node.text + '\n\n'
    return res

In [48]:
retrieved_docs

[NodeWithScore(node=TextNode(id_='13c01a0c-a6e3-4443-bce9-687c2c472f2a', embedding=None, metadata={'file_path': 'D:\\ai_ztf\\fastapi_project\\data\\地方法院.txt', 'file_name': '地方法院.txt', 'file_type': 'text/plain', 'file_size': 4656, 'creation_date': '2025-06-15', 'last_modified_date': '2025-06-12', 'original_doc_id': 'db196138-650f-4c06-832d-f31b9d8ee8da', 'doc_id': 'db196138-650f-4c06-832d-f31b9d8ee8da'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='db196138-650f-4c06-832d-f31b9d8ee8da', node_type='4', metadata={'file_path': 'D:\\ai_ztf\\fastapi_project\\data\\地方法院.txt', 'file_name': '地方法院.txt', 'file_type': 'text/plain', 'file_size': 4656, 'creation_date': '2025-06-15', 'last_modified_date': '2025-06-12',

In [50]:
for nodeWithScore in retrieved_docs:
    print(nodeWithScore.node.text)

到了地方法院之后，我和章先生被几个法警押进待审室。待审室有三四个大房间，用木板隔开的。我和章先生是隔离着的，各人关在一个大房间里面。房里四周用木板搭成长凳，门上有个四方形的洞穴，外面的人可从这个方洞向内望。里面很陈旧龌龊，我一踏进门口，就觉得尿气熏鼻，臭不可当。我于诧异间仔细向四周望望，原来房里摆着一个大马桶，其大无比，好像寻常人家用的大米桶。等一会儿，有个青年法警跑到方洞口张张望望。我看他很和善的样子，便和他谈了几句话，知道他们弟兄们都是很窘苦的，每月九块钱，一切在内，个人还不够用，养家更不得了。又等一会儿，又有一个人在方洞口张望，轻声问我是不是某先生，我说是，略谈之下，才知道他也是我的读者，在法院里任职员，正在吃晚饭，听说我来了，连饭都不吃，特跑来安慰我。他的办公时间原已完了，因为我来，一定要等我审完，好好招呼我进了看守所才肯回去。我说他一天辛苦，要回家休息，不必等我。他不肯，直等到一切布置妥当后才肯离开。
约等了半小时模样，传审了，审得很简单，照例由检察官问姓名、籍贯、年龄、住址等等之后，问起参加救国会的事，问起救国会的宗旨，问起有没有参加煽动日本纱厂工潮。审后再押回待审室坐了一些时候，便押往看守所羁押。
总算优待，原来八个人住的一间囚室，经他们撤清打扫之后，由我和章先生两人同住。房里有一张大木桌可以放些零物。电灯一盏高高地装在两个房间的隔墙上面的方洞里，夜里在房里是看不出什么的，我们到九点钟就安排睡觉，怕有臭虫来光顾，特把木榻搬得离墙远远的。最使我注意的是在那样一个小房间里，也排有待审室里同样大的一个大马桶!第二早起身后，照例打手印，我们这时已打得更熟练了，打出的印子似乎已有了相当的进步!打完了手印，还要一同到天井里去照—张相片。这张相片照得怪有趣!我和章先生各人坐在一只骨牌凳上，列在一排照的；每人坐着，腹前还放着一块石板(好像小学生做算学用的，黑色，上面可用粉笔写字),我的那块石板上面写着雪白的几个大字是“第三千几十号某某(区区的名字)妨碍秩序嫌疑”,章先生的那块石板上面写的相同，不过号码和姓名改一改就是了。我们昂然同坐着照了一张这样的相片。事后我对章先生说，最好能讨得一张取来放大，将来可以高高地悬在客厅里面，让许多朋友“瞻仰瞻仰”,倒是一种很好的纪念。
我们在上海特区第二监狱里关了两天。到廿六日那一天下午，我们在四点半后照例“归号”,铁格子门锁上了，我们

In [52]:

res = ''
for nodeWithScore in retrieved_docs:
    if nodeWithScore.node.metadata['original_doc_id'] in selected_doc_ids:
        res += nodeWithScore.node.text + '\n\n'
print(res)

到了地方法院之后，我和章先生被几个法警押进待审室。待审室有三四个大房间，用木板隔开的。我和章先生是隔离着的，各人关在一个大房间里面。房里四周用木板搭成长凳，门上有个四方形的洞穴，外面的人可从这个方洞向内望。里面很陈旧龌龊，我一踏进门口，就觉得尿气熏鼻，臭不可当。我于诧异间仔细向四周望望，原来房里摆着一个大马桶，其大无比，好像寻常人家用的大米桶。等一会儿，有个青年法警跑到方洞口张张望望。我看他很和善的样子，便和他谈了几句话，知道他们弟兄们都是很窘苦的，每月九块钱，一切在内，个人还不够用，养家更不得了。又等一会儿，又有一个人在方洞口张望，轻声问我是不是某先生，我说是，略谈之下，才知道他也是我的读者，在法院里任职员，正在吃晚饭，听说我来了，连饭都不吃，特跑来安慰我。他的办公时间原已完了，因为我来，一定要等我审完，好好招呼我进了看守所才肯回去。我说他一天辛苦，要回家休息，不必等我。他不肯，直等到一切布置妥当后才肯离开。
约等了半小时模样，传审了，审得很简单，照例由检察官问姓名、籍贯、年龄、住址等等之后，问起参加救国会的事，问起救国会的宗旨，问起有没有参加煽动日本纱厂工潮。审后再押回待审室坐了一些时候，便押往看守所羁押。

总算优待，原来八个人住的一间囚室，经他们撤清打扫之后，由我和章先生两人同住。房里有一张大木桌可以放些零物。电灯一盏高高地装在两个房间的隔墙上面的方洞里，夜里在房里是看不出什么的，我们到九点钟就安排睡觉，怕有臭虫来光顾，特把木榻搬得离墙远远的。最使我注意的是在那样一个小房间里，也排有待审室里同样大的一个大马桶!第二早起身后，照例打手印，我们这时已打得更熟练了，打出的印子似乎已有了相当的进步!打完了手印，还要一同到天井里去照—张相片。这张相片照得怪有趣!我和章先生各人坐在一只骨牌凳上，列在一排照的；每人坐着，腹前还放着一块石板(好像小学生做算学用的，黑色，上面可用粉笔写字),我的那块石板上面写着雪白的几个大字是“第三千几十号某某(区区的名字)妨碍秩序嫌疑”,章先生的那块石板上面写的相同，不过号码和姓名改一改就是了。我们昂然同坐着照了一张这样的相片。事后我对章先生说，最好能讨得一张取来放大，将来可以高高地悬在客厅里面，让许多朋友“瞻仰瞻仰”,倒是一种很好的纪念。

我们在上海特区第二监狱里关了两天。到廿六日那一天下午，我们在四点半后照例“归号”,铁格子门锁上了，

In [ ]:
## 从简单索引中,根据用户问题，获取相关文档,拼接为一个字符串输出
# def get_node_info(node):
#     """
#     获取节点的文档ID和文本内容
    
#     Args:
#         node: 节点对象
        
#     Returns:
#         dict: 包含文档ID和文本内容的字典
#     """
#     return {
#         'doc_id': node.ref_doc_id,
#         'text': node.text
#     }

def get_docs_from_simpleindex(simpleindex,user_question):
    # 创建检索器，检索器只能检索信息，不需要进一步创建搜索或对话引擎
    retriever = simpleindex.as_retriever(similarity_top_k=5)
    # 获取相关文档
    retrieved_docs = retriever.retrieve(user_question)
    result = ''
    # 打印检索到的文档
    for doc in retrieved_docs:
        result += doc.text + '\n'
    return result

res = ''
for nodeWithScore in retrieved_docs:
    if nodeWithScore.node.metadata['original_doc_id'] in selected_doc_ids:
        res += nodeWithScore.node.text + '\n\n'
print(res)

In [44]:
# 第 2 步：节点索引细检，并过滤到第 1 步筛出的文档
node_resp = index.query(
    "作者和章先生在地方法院经历了什么？",
    similarity_top_k=5,
    # llama_index 支持按 metadata 过滤
    doc_ids=selected_doc_ids,
)
# 返回最终节点列表
final_nodes = node_resp.nodes

AttributeError: 'VectorStoreIndex' object has no attribute 'query'

In [25]:
# 第 1 步：摘要索引选文档
summary_resp = summary_index.query("作者和章先生在地方法院经历了什么？", similarity_top_k=1)
selected_doc_ids = [
    n.metadata["doc_id"] for n in summary_resp.nodes
]
selected_doc_ids

# # 第 2 步：节点索引细检，并过滤到第 1 步筛出的文档
# node_resp = node_index.query(
#     "你的问题",
#     similarity_top_k=5,
#     # llama_index 支持按 metadata 过滤
#     doc_ids=selected_doc_ids,
# )
# # 返回最终节点列表
# final_nodes = node_resp.nodes

AttributeError: 'DocumentSummaryIndex' object has no attribute 'query'

In [ ]:

# ##从文档中创建简单索引
# def generate_index(vector_store,input_dir="txt_split_file"):
#     # 读取所有文本文件
#     documents = SimpleDirectoryReader(
#         input_dir=input_dir,
#         recursive=True
#     ).load_data()

#     # 创建索引时直接使用 Settings
#     simpleindex = VectorStoreIndex.from_documents(
#         documents,
#         vector_store=vector_store
#     )
#     return documents,simpleindex

# ## 从简单索引中,根据用户问题，获取相关文档,拼接为一个字符串输出
# def get_docs_from_simpleindex(simpleindex,user_question):
#     # 创建检索器，检索器只能检索信息，不需要进一步创建搜索或对话引擎
#     retriever = simpleindex.as_retriever(similarity_top_k=3)
#     # 获取相关文档
#     retrieved_docs = retriever.retrieve(user_question)
#     result = ''
#     # 打印检索到的文档
#     for doc in retrieved_docs:
#         result += doc.text + '\n'
#     return result

# ##创建摘要索引，自定义摘要内容
# def generate_summary_index(douments):
    
#     summary_tmpl = PromptTemplate(
#     "请用1-2句话总结以下文档，并提出1-2个该文档能回答的用户问题：\n"
#     "{context_str}\n"
#     "—— 摘要：")
#     # LlamaIndex ≥0.10.x 把 summary_template 收进 response_synthesizer
#     resp_synth = get_response_synthesizer(
#         response_mode="tree_summarize",        # or "tree_summarize"
#         summary_template=summary_tmpl
#     )
#     # 构建 DocumentSummaryIndex
#     doc_sum_index = DocumentSummaryIndex.from_documents(
#         # # 读取原始文档（示例用目录，也可以直接传入已构造的 Document 列表）
#         # docs = SimpleDirectoryReader("txt_split_file/").load_data()
#         douments,
#         response_synthesizer=resp_synth
#     )
#     return doc_sum_index

# ##打印每篇文档对应的摘要
# def show_doc_summaries(index: DocumentSummaryIndex):
#     """打印索引中每篇文档的摘要"""
#     struct = index.index_struct                 # IndexDocumentSummary 对象
#     ds = index.docstore

#     for doc_id, summary_node_id in struct.doc_id_to_summary_id.items():
#         summary_node = ds.get_node(summary_node_id)
#         print(f"\n=== 文档 {doc_id} 的摘要 ===")
#         print(summary_node.get_text())

# ##根据用户提问，使用摘要索引获得相关文档
# def get_docs_from_summaryindex(doc_sum_index,query):
#     retriever = doc_sum_index.as_retriever(similarity_top_k=3)
#     # 手动进行检索（结果为 NodeWithScore 对象）
#     retrieved_nodes = retriever.retrieve(query)
#     result = ''
#     # 输出原文内容
#     for i, node in enumerate(retrieved_nodes):
#         result += node.node.text + '\n'
#     return result

# def start_llama_query(deepseekapi):
#     #split_txt_files()
#     llm,embed_model,chroma_collection,vector_store=initialize_llamaindex(deepseekapi=deepseekapi)
#     documents,simpleindex = generate_index(vector_store=vector_store)
#     doc_sum_index = generate_summary_index(documents)
#     return simpleindex,doc_sum_index

In [2]:
def split_markdown_by_heading(markdown_file: str, output_dir: str):
    """
    将markdown文件按一级标题分割成多个txt文件
    
    Args:
        markdown_file: markdown文件路径
        output_dir: 输出目录路径
    """
    import os
    from pathlib import Path
    
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 读取markdown文件
    with open(markdown_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 按一级标题分割
    sections = content.split('# ')
    
    # 跳过第一个空部分
    for section in sections[1:]:
        # 获取标题和内容
        lines = section.split('\n', 1)
        if len(lines) == 2:
            title, content = lines
            # 清理标题中的特殊字符
            title = title.strip().replace('/', '_').replace('\\', '_')
            
            # 写入txt文件
            output_file = os.path.join(output_dir, f"{title}.txt")
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(content.strip())
                
    print(f"文件已分割完成，保存在: {output_dir}")

# 使用示例
split_markdown_by_heading("D:\\ai_ztf_resouce\\reading_material\\markdown.md", "D:\\ai_ztf_resouce\\txt_files")


文件已分割完成，保存在: D:\ai_ztf_resouce\txt_files


'从上次所谈的情形，已可看出《生活》周刊的创办并没有什么大宗的开办费。寥若晨星的职员三个，徐先生月薪二十几块钱，孙先生月薪几块钱，我算是主持全部的事业，月薪最多的了，每月拿六十块钱。我还记得当时在辣斐德路一个小小的过街楼，排了三张办公桌就已觉得满满的，那就是我们的编辑部，也就是我们的总务部，也就是我们的发行部，也就是我们的广告部，也就是我们的会议厅！我们没有大宗的经费，也没有什么高楼大厦。我们有的是几个“患难同事”的心血和努力的精神！我们有的是突飞猛进的多数读者的同情和赞助！《生活》周刊就在这种“心血”、“努力”、“同情”和“赞助”所造成的摇篮里长大起来的。\n\n我永远不能忘记在那个小小的过街楼里，在几盏悬挂在办公桌上的电灯光下面，和徐、孙两先生共同工作到午夜的景象。在那样静寂的夜里，就好像全世界上只有着我们这三个人；但同时念到我们的精神是和无数万的读者联系着，又好像我们是夹在无数万的好友丛中工作着！我们在办公的时候，也往往就是会议的时候；各人有什么新的意思，立刻就提出，就讨论，就议决，就实行！孙先生是偏重于主持会计的事情，虽则他对发行方面也很努力。徐先生是偏重于营业和广告的事情，虽则他在总务方面也很重要。在编辑方面他常用“吟秋”的笔名作些漫画凑凑热闹，因为他不但在营业和广告方面富有创造的天才，而且也对于美术具有深切的兴趣。我的工作当然偏重于编辑和著述方面。我不愿有一字或一句为我所不懂的，或为我所觉得不称心的，就随便付排。校样也完全由我一人看，看校样时的聚精会神，就和在写作的时候一样，因为我的目的要使它没有一个错字；一个错字都没有，在实际上也许做不到，但是我总是要以此为鹄的，至少能使它的错字极少。每期校样要看三次。有的时候，简直不仅是校，竟是重新修正了一下。讲到这里，我还要附带谢谢当时承印我们这个周刊的交通印刷所，尤其是当时在这个印刷所里服务的张铭宝先生和陈锡麟先生。他们不但不怪我的麻烦，而且都成了我的好朋友。\n\n读者一天天多起来，国内外的来信也一天天多起来。我每天差不多要用全个半天来看信。这也是一件极有兴味的工作，因为这就好像天天和许多好友谈话，静心倾听许多读者好友的衷情。其中有一小部分的信是可以在周刊上公开发表和解答的，有大部分的信却有直接答复的必要。有的信虽不能发表，我也用全副精神答复；直接寄去的答复，最长的也有达数千字的。这虽使我感到工作上的极愉快的兴

In [3]:
with open("D:\\ai_ztf_resouce\\txt_files\\二八 一个小小的过街楼.txt",encoding="utf-8") as f:
    text = f.read()
# 打印所有特殊字符
import re

# 使用正则表达式找出所有非字母数字字符
special_chars = re.findall(r'[^\w\s]', text)

# 去重并打印
print("文档中的特殊字符:")
print(set(special_chars))
print(text)

从上次所谈的情形，已可看出《生活》周刊的创办并没有什么大宗的开办费。寥若晨星的职员三个，徐先生月薪二十几块钱，孙先生月薪几块钱，我算是主持全部的事业，月薪最多的了，每月拿六十块钱。我还记得当时在辣斐德路一个小小的过街楼，排了三张办公桌就已觉得满满的，那就是我们的编辑部，也就是我们的总务部，也就是我们的发行部，也就是我们的广告部，也就是我们的会议厅！我们没有大宗的经费，也没有什么高楼大厦。我们有的是几个“患难同事”的心血和努力的精神！我们有的是突飞猛进的多数读者的同情和赞助！《生活》周刊就在这种“心血”、“努力”、“同情”和“赞助”所造成的摇篮里长大起来的。

我永远不能忘记在那个小小的过街楼里，在几盏悬挂在办公桌上的电灯光下面，和徐、孙两先生共同工作到午夜的景象。在那样静寂的夜里，就好像全世界上只有着我们这三个人；但同时念到我们的精神是和无数万的读者联系着，又好像我们是夹在无数万的好友丛中工作着！我们在办公的时候，也往往就是会议的时候；各人有什么新的意思，立刻就提出，就讨论，就议决，就实行！孙先生是偏重于主持会计的事情，虽则他对发行方面也很努力。徐先生是偏重于营业和广告的事情，虽则他在总务方面也很重要。在编辑方面他常用“吟秋”的笔名作些漫画凑凑热闹，因为他不但在营业和广告方面富有创造的天才，而且也对于美术具有深切的兴趣。我的工作当然偏重于编辑和著述方面。我不愿有一字或一句为我所不懂的，或为我所觉得不称心的，就随便付排。校样也完全由我一人看，看校样时的聚精会神，就和在写作的时候一样，因为我的目的要使它没有一个错字；一个错字都没有，在实际上也许做不到，但是我总是要以此为鹄的，至少能使它的错字极少。每期校样要看三次。有的时候，简直不仅是校，竟是重新修正了一下。讲到这里，我还要附带谢谢当时承印我们这个周刊的交通印刷所，尤其是当时在这个印刷所里服务的张铭宝先生和陈锡麟先生。他们不但不怪我的麻烦，而且都成了我的好朋友。

读者一天天多起来，国内外的来信也一天天多起来。我每天差不多要用全个半天来看信。这也是一件极有兴味的工作，因为这就好像天天和许多好友谈话，静心倾听许多读者好友的衷情。其中有一小部分的信是可以在周刊上公开发表和解答的，有大部分的信却有直接答复的必要。有的信虽不能发表，我也用全副精神答复；直接寄去的答复，最长的也有达数千字的。这虽使我感到工作上的极愉快的兴趣，乃至无

In [5]:
def find_paragraphs_with_keyword(file_path: str, keyword: str) -> list:
    """
    在文本文件中查找包含关键词的段落
    
    Args:
        file_path: 文本文件路径
        keyword: 要查找的关键词
        
    Returns:
        包含关键词的段落列表
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        # 按\n\n分割成段落
        paragraphs = content.split('\n\n')
        
        # 过滤出包含关键词的段落
        matching_paragraphs = [p.strip() for p in paragraphs if keyword in p]
        
        return matching_paragraphs
        
    except Exception as e:
        print(f"发生错误: {str(e)}")
        return []

# 使用示例
file_path = "D:\\ai_ztf_resouce\\txt_files\\二八 一个小小的过街楼.txt"
keyword = "生活"
matching_paragraphs = find_paragraphs_with_keyword(file_path, keyword)

print(f"包含关键词 '{keyword}' 的段落:")
for i, para in enumerate(matching_paragraphs, 1):
    print(f"\n段落 {i}:")
    print(para)


包含关键词 '生活' 的段落:

段落 1:
从上次所谈的情形，已可看出《生活》周刊的创办并没有什么大宗的开办费。寥若晨星的职员三个，徐先生月薪二十几块钱，孙先生月薪几块钱，我算是主持全部的事业，月薪最多的了，每月拿六十块钱。我还记得当时在辣斐德路一个小小的过街楼，排了三张办公桌就已觉得满满的，那就是我们的编辑部，也就是我们的总务部，也就是我们的发行部，也就是我们的广告部，也就是我们的会议厅！我们没有大宗的经费，也没有什么高楼大厦。我们有的是几个“患难同事”的心血和努力的精神！我们有的是突飞猛进的多数读者的同情和赞助！《生活》周刊就在这种“心血”、“努力”、“同情”和“赞助”所造成的摇篮里长大起来的。

段落 2:
读者一天天多起来，国内外的来信也一天天多起来。我每天差不多要用全个半天来看信。这也是一件极有兴味的工作，因为这就好像天天和许多好友谈话，静心倾听许多读者好友的衷情。其中有一小部分的信是可以在周刊上公开发表和解答的，有大部分的信却有直接答复的必要。有的信虽不能发表，我也用全副精神答复；直接寄去的答复，最长的也有达数千字的。这虽使我感到工作上的极愉快的兴趣，乃至无上的荣幸，但是时间却渐渐不够起来了，因此只得摆脱一切原有的兼职，日夜都做《生活》周刊的事情，做到深夜还舍不得走。我的妻有一次和我说笑话，她说：“我看你恨不得要把床铺搬到办公室里面去！”其实后来纵然“把床铺搬到办公室里面去”也是来不及的。后来最盛的时候，有五六个同事全天为着信件的事帮我的忙，还有时来不及，一个人纵然不睡觉也干不了！

段落 3:
但是《生活》周刊的发展是随着本身经济力的发展而逐渐向前推的，所以在增加职员方面不得不慢慢儿来，因此事务的增繁和人手的增多，常常不能成正比例。《生活》周刊本身经济力的发展，来源不外两方面：一方面是发行的推广，由此增加报费的收入；一方面是广告费的收入随着销数的增加而增加。我们既没有什么大宗的经费，事业的规模不得不看这两方面的收入做进行的根据，因为我们是要量入为出的；但是我们所欣幸的，是我们可以尽量运用我们在这两方面的收入，扩充我们的事业，没有什么“老板”在后面剥削我们。关于这一点，我们不得不感谢职教社。当时《生活》周刊还在职教社的“幪”之下，我和徐孙诸先生都只是雇员，原没有支配的全权，但是职教社当局的诸先生全把这件事看作文化事业，一点没有从中取利的意思。


In [6]:
def extract_keywords(query: str, top_k: int = 5) -> list:
    """
    从用户查询中提取关键词
    
    Args:
        query: 用户输入的查询文本
        top_k: 返回的关键词数量
        
    Returns:
        关键词列表
    """
    try:
        # 使用 jieba 分词
        import jieba
        import jieba.analyse
        
        # 使用 TF-IDF 算法提取关键词
        keywords = jieba.analyse.extract_tags(query, topK=top_k)
        
        return keywords
        
    except Exception as e:
        print(f"提取关键词时发生错误: {str(e)}")
        return []

# 使用示例
query = "作者和章先生在地方法院经历了什么？"
keywords = extract_keywords(query)
print(f"从查询中提取的关键词: {keywords}")


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\kiwid\AppData\Local\Temp\jieba.cache
Loading model cost 0.810 seconds.
Prefix dict has been built successfully.


从查询中提取的关键词: ['地方法院', '作者', '经历', '先生', '什么']


In [ ]:
def get_user_keywords(chat_history):
    from llama_index.llms.deepseek import DeepSeek
    import json
    llm = DeepSeek(
            api_key="sk-5f2880952eb543a59d02d1015dcdd8e1",  # 替换为你的 DeepSeek API key
            model="deepseek-chat"
        )
        # 使用llm进行查询

    chat_history=messages=[
            {"role": "assistant", "content": "你好"},
            {"role": "user", "content": "邹韬奋是谁？"},
            {"role": "assistant", "content": "他是一个伟大的作家"},
            {"role": "user", "content": "你吃了么"},
        ]
    GET_USER_KEYWORDS="""
    以下是AI和用户的部分聊天记录，用户可能在和AI（assistant）讨论邹韬奋的有关信息，也有可能并不是和AI讨论邹韬奋的有关信息。
    请根据聊天记录判断，聊天记录中最后一句用户发言是否围绕邹韬奋在讨论，输出布尔字段：ture或者false。
    如果是，则进一步提取用户发言中的关键词，输出关键词列表。关键词将用于在一个邹韬奋作品数据库中进行搜索。
    答案以json格式输出，json格式为：{"is_about_ztaofen":true,"keywords":["关键词1","关键词2","关键词3"]}
    聊天记录是：\n\n
    {chat_history}
    """
    GET_USER_KEYWORDS = GET_USER_KEYWORDS.replace("{chat_history}",str(chat_history))
    try:
        response = llm.complete(GET_USER_KEYWORDS)
        json_str = response.text.strip('```json\n').strip('```')
        json_data = json.loads(json_str)
        return json_data
    except Exception as e:
        return {"is_about_ztaofen":False,"keywords":[]}



In [24]:

print(GET_USER_KEYWORDS)


以下是AI和用户的部分聊天记录，用户可能在和AI（assistant）讨论邹韬奋的有关信息，也有可能并不是和AI讨论邹韬奋的有关信息。
请根据聊天记录判断，聊天记录中最后一句用户发言是否围绕邹韬奋在讨论，输出布尔字段：ture或者false。
如果是，则进一步提取用户发言中的关键词，输出关键词列表。关键词将用于在一个邹韬奋作品数据库中进行搜索。
答案以json格式输出，json格式为：{"is_about_ztaofen":true,"keywords":["关键词1","关键词2","关键词3"]}
聊天记录是：


[{'role': 'assistant', 'content': '你好'}, {'role': 'user', 'content': '邹韬奋是谁？'}, {'role': 'assistant', 'content': '他是一个伟大的作家'}, {'role': 'user', 'content': '你吃了么'}]



In [ ]:
# 使用 requests 库发送 GET 请求到 Elasticsearch,获取k个文档
def get_es_docs(kw,k):
    import requests,json
    # 构建查询 URL
    url = "http://localhost:9200/article/_search"
    params = {
        "q": "body:kw",
        "size": k,
        "pretty": "true"
    }
    # 发送 GET 请求
    response = requests.get(url, params=params)
    body_list = []
    res_list = json.loads(response.text)['hits']['hits']
    for res in res_list:
        body_list.append(res['_source']['body'])
    return body_list


['廿三日那天从黎明到下午三点钟（这时间是当时向旁人探问才知道的，我身上没有表)，为时不能算久，但是在我却好像过了好久的时候，因为带着一对“半瞎”的眼睛，拖着一双没有带子的皮鞋，下身穿着一条没有裤带常常下落的里裤，躑躅兜转了无数次的楼梯、天井、走廊；走廊、天井、楼梯！到了下午三点钟左右，我又由囚室里被提了出来，和章先生史女士同被几个巡捕和法院的法警押到高三分院去。将押出门的时候，史女士先走，我和章先生随在后面，有个法国人用手铐把我的右手臂和章先生的左手臂套在一起，把锁锁上，所以我们两个人不得不并排走。套手铐也是我生平第一次的经验。我突然被套上手铐的刹那间，在脑际所闪过的奇特的感觉，和第一次打手印时一样，觉得这是使我不胜愤怒的侮辱，但想我所以受到这样的侮辱是因为我努力参加救国运动，我应该把这愤怒转变为继续奋斗的力量。我一面这样想着，一面昂首挺起胸膛大踏步走一—虽则脚上拖着没有带子的皮鞋，大踏步是格外费力的。捕房离法院很近，不过离开几家的路。我们出了捕房的大门，走过一段马路就到了法院。在马路上走的时候，前后拥着巡捕和法警，还有外国侦探，路人都停住脚，用奇异的眼光看着我们。我们到法院后，和史女士同到待审室里面去。往待审室上楼梯的时候，已有亲友数十人在旁拥聚着等候我们。我们进了待审室后，我和章先生的手铐被开了锁，脱了下来。待审室外面还有一个房间连着，那里有几个法警是被派来监视我们的，但是他们都已知道我们是为着主张团结救国而犯罪的，对于我们表示着很恳切的同情，说“你们的意思，做中国人的谁不赞成！”法警室的外面便是走进法庭的走廊，门是常常关着，偶然开一下，便有亲友们在外面伸着头遥望着，可是我们还不能见面谈话。他们送进来一包水果和饼干，我们三个人这时都觉得饿了，便吃了一些。在这待审室里，我们三个人都可以随便谈话，各人彼此告诉了前一夜被捕的经过。他们两位都是在深夜三点钟左右被捕的。我们三个人都住在法租界，所以都被捕到法捕房来。等一会儿，由外面传进的消息，说前一夜在公共租界被捕的沈、章、王、沙四先生于当日上午十点钟经高二分院开审后，于当天十二点钟即由各人的律师保了出来。我们听了，都觉得快慰。正谈论间，法警室的门又偶然开了一下，章先生瞥见沈先生在门外笑着举手向我们招呼，章先生即对我们笑着欢呼：“沈先生来了！沈先生来了！”我赶紧转眼看时，门又关了，我虽看不见沈先生，但是想到沈先生自己午后

In [41]:
res_list

[{'_index': 'article',
  '_type': '_doc',
  '_id': 'mTZ_c5cB4fdKsnzVIlKW',
  '_score': 3.0388975,
  '_source': {'@version': '1',
   'updated_at': '2025-06-15T11:20:44.979732Z',
   'id': 51,
   '@timestamp': '2025-06-15T12:10:00.907875200Z',
   'title': '三六 高三分院.txt',
   'body': '廿三日那天从黎明到下午三点钟（这时间是当时向旁人探问才知道的，我身上没有表)，为时不能算久，但是在我却好像过了好久的时候，因为带着一对“半瞎”的眼睛，拖着一双没有带子的皮鞋，下身穿着一条没有裤带常常下落的里裤，躑躅兜转了无数次的楼梯、天井、走廊；走廊、天井、楼梯！到了下午三点钟左右，我又由囚室里被提了出来，和章先生史女士同被几个巡捕和法院的法警押到高三分院去。将押出门的时候，史女士先走，我和章先生随在后面，有个法国人用手铐把我的右手臂和章先生的左手臂套在一起，把锁锁上，所以我们两个人不得不并排走。套手铐也是我生平第一次的经验。我突然被套上手铐的刹那间，在脑际所闪过的奇特的感觉，和第一次打手印时一样，觉得这是使我不胜愤怒的侮辱，但想我所以受到这样的侮辱是因为我努力参加救国运动，我应该把这愤怒转变为继续奋斗的力量。我一面这样想着，一面昂首挺起胸膛大踏步走一—虽则脚上拖着没有带子的皮鞋，大踏步是格外费力的。捕房离法院很近，不过离开几家的路。我们出了捕房的大门，走过一段马路就到了法院。在马路上走的时候，前后拥着巡捕和法警，还有外国侦探，路人都停住脚，用奇异的眼光看着我们。我们到法院后，和史女士同到待审室里面去。往待审室上楼梯的时候，已有亲友数十人在旁拥聚着等候我们。我们进了待审室后，我和章先生的手铐被开了锁，脱了下来。待审室外面还有一个房间连着，那里有几个法警是被派来监视我们的，但是他们都已知道我们是为着主张团结救国而犯罪的，对于我们表示着很恳切的同情，说“你们的意思，做中国人的谁不赞成！”法警室的外面便是走进法庭的走廊，门是常常关着，偶然开一下，便有亲友们在外面伸着头遥望着，可是我们还不能见面谈话。他们送进来一包水果和饼干

In [ ]:
GET http://localhost:9200/my_index/_search?q=status:active&size=10&from=0&sort=timestamp:desc&pretty=true



CompletionResponse(text='```json\n{\n  "is_about_ztaofen": false,\n  "keywords": []\n}\n```', additional_kwargs={'prompt_tokens': 195, 'completion_tokens': 22, 'total_tokens': 217}, raw=ChatCompletion(id='1e2fdbaa-77ec-4976-a477-653cd6bc2169', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "is_about_ztaofen": false,\n  "keywords": []\n}\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1749988228, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_8802369eaa_prod0425fp8', usage=CompletionUsage(completion_tokens=22, prompt_tokens=195, total_tokens=217, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=128), prompt_cache_hit_tokens=128, prompt_cache_miss_tokens=67)), logprobs=None, delta=None)

In [29]:
from elasticsearch import Elasticsearch, helpers

# 1. 连接到本地 ES
es = Elasticsearch("http://localhost:9200")

# 2. 执行查询
response = es.search(
    index="article",
    body={
        "query": {
            "term": {"body": "章"}
        },
        "size": 5
    }
)

# 3. 处理结果
print(f"总命中文档数：{response['hits']['total']['value']}")
for hit in response["hits"]["hits"]:
    print(hit["_source"])


UnsupportedProductError: The client noticed that the server is not Elasticsearch and we do not support this unknown product

In [1]:
from util.chat_util import initialize_llamaindex,get_user_keywords,get_es_docs,find_paragraphs_with_keyword
from util import db_util
from util.db_util import execute_query  #连接pg数据库并进行查询

#启动es,默认9200端口
#cd D:\elastic\elasticsearch-9.0.2\bin
#.\elasticsearch.bat

# cd D:\develop\elasticSearch\logstash-9.0.2\bin
# # 前台启动并指定配置文件
# .\logstash.bat -f "D:\\develop\\elasticSearch\\logstash-9.0.2\\pg2es.conf" 用绝对路径写conf
#或者 .\logstash.bat -f "../pg2es.conf" -w 设置守护进程


#初始化设置llm和embed模型
#llm,embed_model = initialize_llamaindex(deepseekapi="sk-5f2880952eb543a59d02d1015dcdd8e1")
#载入两个索引
# summary_index,simple_index = db_util.load_indexes()
# res = db_util.get_final_nodes_text(summary_index,simple_index,user_query="马桶在哪里？")
# user_keywords = get_user_keywords()
# print(user_keywords)
# print('res',res)
chat_history=[
            {"role": "assistant", "content": "你好"},
            {"role": "user", "content": "邹韬奋是谁？"},
            {"role": "assistant", "content": "他是一个伟大的作家"},
            {"role": "user", "content": "邹韬奋和生活是什么关系？"},

        ]
an_kw = get_user_keywords(chat_history)
print('an_kw',an_kw)
print('*****************')
if an_kw['is_about_ztaofen']:
    kw_paragraphs_list = []  # 期望得到: [ [邹韬奋的段落], [生活的段落] ]
    
    for kw in an_kw['keywords']:
        docs = get_es_docs(kw)
        
        # 这是为当前关键词找到的所有段落
        paragraphs_for_this_kw = [] 
        
        for doc in docs:
            paragraphs = find_paragraphs_with_keyword(doc, kw,n=5)
            #不是append，因此不会产生嵌套列表
            paragraphs_for_this_kw.extend(paragraphs)
            
        # 修正：将当前关键词的段落列表添加到最终结果中
        kw_paragraphs_list.append(paragraphs_for_this_kw)
        
    print(kw_paragraphs_list)


d:\python_envs\ai_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


an_kw {'is_about_ztaofen': True, 'keywords': ['邹韬奋', '生活', '关系']}
*****************
[['I.①韬…Ⅱ.①生…Ⅲ.①邹韬奋（1895\\~1944） 传记IV. $\\textcircled{1}$ K825.42', '一九四四年七月二十四日早晨七时二十分，伟大的爱国者、民主战士邹韬奋同志与世长辞。终年四十九岁。这是他在上海医院的最后遗容。'], ['在市人路旅合障  \n丁行，理有用， O 餐理理與實践 韬奋以他一贯的真A 热情，将此信在《生活星  \n5 刊》上全文刊载。在编者的话中，对刘少奇同志的主张表示福 热烈赞同，并补充了自己的看生 法。5  \n健 人民陣線與關門主羲 信滋A ： 英文禁 楼 中 前  \nS仕 只有门 有心理留的人们，今日 老淀今 （一）在合歌路之下， 国公武间合股按间，在费询过 者先生 開於救國聯合戟線的畿個疑問  \n饰 面  \n不分的的 福 林且必组估 中共 合', '一九三六年六月十九日刘少奇同志给韬奋写去第二封长信，表达了对创刊十二天的《生活日报》的肯定和希望，并着重阐述了人民阵线与关门主义的问题。', '《生活日报》在香港出版，影响只能偏于东南一隅，且交通不便，不利于全国性的宣传。国内局势的变化，救亡运动的发展也很需要韬奋回到上海工作。他接受党组织的建议，在七月三十一日自动结束《生活日报》，拟迁往上海出版。国民党政府百般阻挠，不允登记，初议未能实现。', '《生活星期刊》第十九号上刊登的韬奋的《“中国与中国人”导言》一文手稿。', '《生活星期刊》原是《生活日报》的星期日增刊。一九三六年八月，韬奋将《生活星期增刊》改名《生活星期刊》迁至上海出版。为了便于读者保存，将原来的八开大本改为十六开本。这是《生活星期刊》第十二号上刊登的韬奋撰写的《不是创刊词》。', '韬奋出身的时代，正是封建专制主义统治我国两千多年，最后一个封建王朝行将崩溃的前夜；也是戊戌政变失败以后，资产阶级民主主义启蒙思想在知识分子阶层中广泛传播的时候。韬奋出身于一个没落的封建官僚大家庭，和他所有同时代人一样，幼年接受的是家塾式的封建教育，辛亥革命以后，则是那个时代特有的改良教育；最后，在大学里接受的是典型的西方资产阶级教育。在中学和大学时期，韬奋完全靠自己的勤勉，过半工半读

In [4]:
len(kw_paragraphs_list[1])

23

In [25]:
kw_paragraphs_list[0][1]

'一九三六年六月十九日刘少奇同志给韬奋写去第二封长信，表达了对创刊十二天的《生活日报》的肯定和希望，并着重阐述了人民阵线与关门主义的问题。'

In [8]:
find_paragraphs_with_keyword("aa\n\nbb\n\ncc\n\naa\n\n","aa")

['aa', 'aa']

In [50]:
get_es_docs('公安')

[]

In [51]:
import requests
kw='公安'
k=5
url = "http://localhost:9200/article/_search"
params = {
    "q": "body:{kw}".format(kw=kw),
    "size": "{k}".format(k=k),
    "pretty": "true"
}
# 发送 GET 请求
response = requests.get(url, params=params)
print('params',params)

params {'q': 'body:公安', 'size': '5', 'pretty': 'true'}


In [52]:
json.loads(response.text)['hits']['hits']

[{'_index': 'article',
  '_type': '_doc',
  '_id': 'gDZ_c5cB4fdKsnzVIlKV',
  '_score': 2.3247056,
  '_source': {'@version': '1',
   'updated_at': '2025-06-15T11:20:51.638568Z',
   'id': 83,
   '@timestamp': '2025-06-15T12:10:00.883208400Z',
   'title': '四〇 押在公安局.txt',
   'body': '在上海地方法院的看守所里只关了一天一夜，廿七日下午六点钟左右，又像老鼠被猫衔在嘴里奔窜着，由地方法院转解到公安局里去。我们先由看守所里被提出来，重新光顾待审室一次，不过这时我和章先生两个人同被押进一间待审室里去等候着。这时的法警对我们比初来的那天客气得多，大概他们都知道了我们是为着参加救国运动才到这里来的。等一会儿，我们先后被传审，检察官虽换了一个，但是问答内容还是上次那同样的一套。不过这次那位审问我们的检察官却充满着同情和歉意，拿出公安局的公文给我们看，里面说要移提我们去和沈先生等四位“对质”。检察官宣布之后，我们也没有什么话说，走出法庭后，就有公安局派来的人员迎上来押着出去。法院里有几个职员赶出来和我们握手送别，我们又觉到爱国的同胞们随处给予我们的同情和厚意的可感。我无意中和章先生说出了这一句话，在旁边同走的那位公安局的科员插着说：“这是各位先生人格的感动。”我说：“这倒不是我们几个个人的人格问题，却是有许多同胞不愿做亡国奴的心理的流露！”\n\n我们随着一群公安局的侦探、警察和科员，拥出了法院的门口，分乘汽\n\n车直驶公安局而去。\n\n这时沈先生等四位已押在公安局三四天了，我们一进公安局，就被引到他们的房间里去。我们在患难中相见畅谈，当然是格外快慰的，彼此诉说了一番经过的情形，又说了不少互相安慰的话语。我看看他们所住的房间还算清洁，是在二层楼上，前面有个露台，露台的前面是大天井，立在露台上可以看着公安局的大门；房的后面靠壁处并列排着四个小铁床，便是他们四位的床铺；这一列铁床的前面有个屏风遮着，屏风的外面放着一张圆桌、几张椅子，便是我们吃饭和相聚谈话的地方。我踏进房里一会儿之后，觉得奇特的是总有一个不相识的

In [ ]:

uvicorn main:app --reload